In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from env import host, user, password
import zillo_wrangle

In [2]:
# function to contact database
def get_db_url(db_name):
    return f"mysql+pymysql://{user}:{password}@{host}/{db_name}"

In [3]:
# function to query database and return zillow df
def get_zillo():
    query = """
    SELECT bedroomcnt as bedrooms, 
       bathroomcnt as bathrooms,
       calculatedfinishedsquarefeet as square_feet,
       taxvaluedollarcnt as home_value,
       yearbuilt as year,
	   taxamount as taxes,
       fips as fips_number
    FROM predictions_2017
    JOIN properties_2017 USING(id)
    JOIN propertylandusetype USING(propertylandusetypeid)
    WHERE #(transactiondate >= '2017-05-01' AND transactiondate <= '2017-06-30') 
        propertylandusetypeid = '261'
        AND bedroomcnt > 0
        AND bathroomcnt > 0
        AND calculatedfinishedsquarefeet > 0 
        AND taxamount > 0
        AND taxvaluedollarcnt > 0
        AND fips > 0
    ORDER BY fips;
    """
    df = pd.read_sql(query, get_db_url('zillow'))
    return df

In [4]:
# function to clean up my zillow df
def clean_data(df):
    '''
    This funciton takes in the zillow df and drops observations with Null values
    and handles data types returning a df with a basic clean.
    '''
    df = df.dropna()
    df['fips_number'] = df['fips_number'].astype(int)
    df['square_feet'] = df['square_feet'].astype(int)
    df['year'] = df['year'].astype(int)

    return df

In [5]:
# composite
def wrangle_zillow():
    df = get_data_from_sql()
    df = clean_data(df)
    return df 

In [6]:
zillo = zillo_wrangle.wrangle_zillow()

/Users/jaredgodar/codeup-data-science/regression-exercises/zillo_wrangle.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips_number'] = df['fips_number'].astype(int)
/Users/jaredgodar/codeup-data-science/regression-exercises/zillo_wrangle.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['square_feet'] = df['square_feet'].astype(int)
/Users/jaredgodar/codeup-data-science/regression-exercises/zillo_wrangle.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [7]:
zillo.describe().T

,count,mean,std,min,25%,50%,75%,max
bedrooms,55588.0,3.311380,0.924005,1.00,3.000,3.000,4.00,25.00
bathrooms,55588.0,2.254884,0.992818,1.00,2.000,2.000,3.00,20.00
square_feet,55588.0,1870.208031,964.140876,320.00,1262.000,1631.000,2217.00,26345.00
home_value,55588.0,456879.285745,593725.070535,9762.00,188731.500,330507.000,534667.50,30166843.00
year,55588.0,1961.232496,21.959743,1862.00,1949.000,1958.000,1976.00,2016.00
taxes,55588.0,5566.992804,6827.036130,20.24,2539.715,4139.215,6415.89,342940.38
fips_number,55588.0,6048.770058,21.129554,6037.00,6037.000,6037.000,6059.00,6111.00


In [8]:
zillo.dtypes

bedrooms       float64
bathrooms      float64
square_feet      int64
home_value     float64
year             int64
taxes          float64
fips_number      int64
dtype: object

In [9]:
zillo.shape

(55588, 7)

In [10]:
plt.rc('figure', figsize=(16, 8))

plt.rc('font', size=14)